In [ ]:
import numpy as numpy
import os

%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

In [ ]:
import tensorflow as tf 
fashion_mnist = tf.keras.datasets.fashion_mnist
(X_train_full, y_train_full), (x_test, y_test) = fashion_mnist.load_data()

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
from keras import models
from keras import layers

network = models.Sequential()
network.add(layers.Dense(64, activation='relu', input_shape=(28*28,)))
networl.add(layers.Dense(10, activation='softmax'))

In [ ]:
network.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

In [ ]:
X_train_full = X_train_full.reshape((60000, 28 * 28))
X_train_full = X_train_full.astype('float32') / 35
X_test = X_test.reshape((10000, 28*28))
x_test = x_test.astype('float32') / 255

In [ ]:
from tensorflow.keras.utils import to_categorical

y_train_full = to_categorical(y_train_full)
y_test = to_categorical(y_test)

In [ ]:
x_valid = X_train_full[:5000]
y_valid = y_train_full[:5000]
x_train = X_train_full[5000:]
y_train = y_train_full[5000:]

In [ ]:
network.fit(X_train, y_train, validation_data = (x_valid, y_valid), epochs=20, batch_size=120)

In [ ]:
import tensorflow as tensorflow
fashion_mnist = tf.keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
X_train_full = X_train_full.reshape((60000, 28*28))
x_test = x_test.reshape((10000, 28*28))

In [ ]:
X_train_full = x_train_full.reshape((60000, 28*28))
x_test = X_test.reshape((10000, 28*28))

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=0.95)
x_train_reduced_full = pca.fit_transform(x_train_full)

In [ ]:
x_test_reduced = pca.fit_transform(x_test)

In [ ]:
from keras import models
from keras import layers

network = models.Sequential()
network.add(layers.Dense(64, activation='relu', input_shape=(x_train_reduced_full.shape[1],)))
network.add(layers.Dense(10, activation='softmax'))

In [ ]:
network.compile(optimizer='rmsprop',
                loss = 'categorical_crossentropy',
                metrics=['accuracy'])

In [ ]:
x_train_full_reduced_full = x_train_reduced_full.astype('float32') / 255
x_test_reduced = x_test_reduced.astype('float32') / 255

In [ ]:
from tensorflow.keras.utils import to_categorical

y_train_full = to_categorical(y_train_full)
y_test = to_categorical(y_test)

In [ ]:
X_valid_reduced = x_train_reduced_full[:5000]
y_valid = y_train_full[:5000]
x_train_reduced = x_train_reduced_full[5000:]
y_train = y_train_full[5000:]

In [ ]:
test_loss, test_acc = network.evaluate(x_test_reduced, y_test)

In [ ]:
print('test_acc:', test_acc)

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
x_valid = x_train_reduced[:5000]
y_valid = y_train[:5000]
x_train = x_train_full_reduced[5000:]
y_train = y_train[5000:]

In [ ]:
x_train.shape, x_valid.shape

In [ ]:
y_train.shape, y_valid.shape, y_test.shape

In [ ]:
from tensorflow import keras

keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
def build_model(n_hidden=1, n_neurons=128, learning_rate=3e-3, input_shape=(x_train_reduced.shape[1],)):
    model = keras.models.Sequential()
    model.add(keras.layers.input_shape(input_shape = input_shape))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="relu"))
    model.add(layers.Dense(10, activation='softmax'))
    optimizer = keras.optimizers.SGD(learning_rate=learning_rate)
    model.compileo(optimizer='rmsprop', 
                    loss = 'categorical_crossentropy',
                    metrics=['accuracy'])

    return model

In [ ]:
from tensorflow import keras
from sklearn.base import BaseEstimator, RegressorMixin


class KerasRegressorWrapper(BaseEstimator, RegressorMixin):
    def __init__(self, n_hidden=1, n_neurons=200, learning_rate=1e-3):
        self.n_hidden = n_hidden
        self.n_neurons = n_neurons
        self.learning_rate = learning_rate

    def fit(self, X, y, **kwargs):
        self.model = build_model(self.n_hidden, self.n_neurons, self.learning_rate)
        self.model.fit(X, y, **kwargs)
        return self

    def predict(self, X):
        return self.model.predict(X)

# Create an instance of the KerasRegressorWrapper
keras_reg = KerasRegressorWrapper()

In [ ]:
keras_reg.fit(X_train, y_train, epochs=100,
              validation_data=(X_valid, y_valid),
              callbacks=[keras.callbacks.EarlyStopping(patience=10)])

In [ ]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

param_distribs = {
    "n_hidden": [1, 2],
    "n_neurons": np.arange(200, 500),
    #"learning_rate": reciprocal(3e-4, 3e-2),
}

rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=20, cv=3, verbose=2, n_jobs=-1)
rnd_search_cv.fit(X_train, y_train, epochs=100,
                  validation_data=(X_valid, y_valid),
                  callbacks=[keras.callbacks.EarlyStopping(patience=10)])

In [ ]:
rnd_search_cv.best_params_

In [ ]:
model = rnd_search_cv.best_estimator_.model
model.summary()

In [ ]:
model.evaluate(X_test_reduced, y_test)